# ASync Example Bibliograpy Explorer

In [1]:
import grpc
import json
import asyncio
from example_bib_app.grpc import example_bib_app_pb2_grpc, example_bib_app_pb2

from datetime import datetime

In [2]:
# gRPC clients

grpc_server_host = 'localhost' # '127.0.0.1' 
grpc_server_port = 50051

sync_channel = grpc.aio.insecure_channel(f'{grpc_server_host}:{grpc_server_port}')
author_client = example_bib_app_pb2_grpc.AuthorControllerStub(sync_channel)
publisher_client = example_bib_app_pb2_grpc.PublisherControllerStub(sync_channel)

book_client = example_bib_app_pb2_grpc.BookControllerStub(sync_channel)

## Creating Authors

In [ ]:

for i in range(10):
    author_response = await author_client.Create(example_bib_app_pb2.AuthorRequest(name_first='John', name_last=f'Doe{i}', birth_date=datetime.now().strftime('%Y-%m-%d')))
    author_response

In [ ]:
res = await author_client.List(example_bib_app_pb2.AuthorListRequest())
res.results
for author in res.results:
    print(author.name_first, author.name_last)


In [ ]:
author_response.author_id

In [ ]:
publisher_response = await publisher_client.Create(example_bib_app_pb2.PublisherRequest(name='Hopper', website='https://hopper.com' ))
publisher_response

In [ ]:
# list publishers
res = await publisher_client.List(example_bib_app_pb2.PublisherListRequest())
res.results

## Using Filters

In [ ]:
filter_as_dict = {"name_last": "Doe"}
metadata = (("filters", (json.dumps(filter_as_dict))),)

res = await author_client.List(example_bib_app_pb2.AuthorListRequest(), metadata=metadata)
res.results


## Exploring Books

In [ ]:
# creating books
for i in range(10):
    await book_client.Create(example_bib_app_pb2.BookRequest(title=f'book {i}', 
                                                            authors=[author_response.author_id],
                                                            isbn=f'isbn-{i}',
                                                            publisher=publisher_response.publisher_id,
                                                            publication_date=datetime.now().strftime('%Y-%m-%d')
                                                            ))

In [ ]:
# list books

res = await book_client.List(example_bib_app_pb2.BookListRequest())
res.results

In [3]:
# stream books

async for book in  book_client.Stream(example_bib_app_pb2.BookStreamRequest()):
    print(book)


AioRpcError: <AioRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "'InternalHttpRequest' object has no attribute 'write'"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"\'InternalHttpRequest\' object has no attribute \'write\'", grpc_status:2, created_time:"2023-11-23T22:15:10.611716914+01:00"}"
>

In [5]:
# stream books
async for book in book_client.StreamBooksByIDList(example_bib_app_pb2.BookStreamBooksByIDListRequest(book_ids = ["d507ea75-a416-43c2-9de5-55a7e732b531", "0f82740f-f600-40a2-b7af-eb0af42ab975"])):
    print(book)

AttributeError: module 'example_bib_app.grpc.example_bib_app_pb2' has no attribute 'BookStreamBooksRequest'

## Closing channels

In [ ]:
sync_channel.close()